# SPARQL 쿼리 실습 💻

## 필요 패키지 설치 및 불러오기 🛒

In [ ]:
!pip install rdflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.2 MB/s eta 0:00:00


In [ ]:
# 필요 패키지 불러오기
import pandas as pd
import re
from pyparsing import ParseException
from rdflib import Graph, URIRef, BNode, Literal, Namespace, plugin
from rdflib.namespace import RDF, OWL, RDFS, XSD
from rdflib.plugins.sparql import prepareQuery
from rdflib.store import Store, NO_STORE, VALID_STORE
from IPython.display import display
from rdflib.graph import Dataset
import sqlite3

In [ ]:
# 그래프 객체 정의
g = Graph()

# 그래프 파싱하여 불러오기
g.parse('/content/kpop.rdf')

<Graph identifier=Nb8d11c659a314d3f8642ebdc1604e7f5 (<class 'rdflib.graph.Graph'>)>

## 기초 SPARQL 쿼리문 🔎

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> -> 긴 URI를 rdf라는 변수로 지정

SELECT ?x -> x를 찾을 것이다

WEHRE {?x rdf:type ?y} -> y라는 개념의 예시인 x  

### Instance 검색 💡 - rdf:type
앨범 예시 찾기

In [ ]:
knows_query = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX kpop: <http://www.semanticweb.org/singer/k-pop/idol#>

SELECT ?group
WHERE{
?group rdf:type kpop:Girl_Group
}
"""
qres = g.query(knows_query)

In [ ]:
qq = []
for row in qres:
  qq.append(list(row))

dff = pd.DataFrame(qq)

dff= dff.replace('http://www.semanticweb.org/singer/k-pop/idol#', '', regex=True)

dff

,0
0,뉴진스
1,르세라핌
2,블랙핑크
3,아이브
4,에스파


### Subclass 검색 💡 - rdfs:subClassOf
그룹의 subclass 찾기

In [ ]:
knows_query = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX kpop: <http://www.semanticweb.org/singer/k-pop/idol#>

SELECT ?group
WHERE{
?group rdfs:subClassOf* kpop:Group
}
"""
qres = g.query(knows_query)

In [ ]:
qq = []
for row in qres:
  qq.append(list(row))

dff = pd.DataFrame(qq)

dff= dff.replace('http://www.semanticweb.org/singer/k-pop/idol#', '', regex=True)

dff

,0
0,Group
1,Boy_Group
2,Girl_Group


### Object Property를 이용한 검색 💡
뉴진스 멤버 찾기

In [ ]:
knows_query = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX kpop: <http://www.semanticweb.org/singer/k-pop/idol#>

SELECT ?newjeans
WHERE{
?newjeans kpop:member_of kpop:뉴진스
}
"""
qres = g.query(knows_query)

In [ ]:
qq = []
for row in qres:
  qq.append(list(row))

dff = pd.DataFrame(qq)

dff= dff.replace('http://www.semanticweb.org/singer/k-pop/idol#', '', regex=True)

dff

,0
0,다니엘
1,민지
2,하니
3,해린
4,혜인


### Datatype property를 이용한 검색 💡
세븐틴의 멤버 수 찾기

In [ ]:
knows_query = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX kpop: <http://www.semanticweb.org/singer/k-pop/idol#>

SELECT ?seventeen
WHERE{
kpop:세븐틴 kpop:number_of_members ?seventeen
}
"""
qres = g.query(knows_query)

In [ ]:
qq = []
for row in qres:
  qq.append(list(row))

dff = pd.DataFrame(qq)

dff= dff.replace('http://www.semanticweb.org/singer/k-pop/idol#', '', regex=True)

dff

,0
0,13


### 복잡한 SPARQL 쿼리 💡

4세대 보이그룹 모드 찾기

In [ ]:
knows_query = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX kpop: <http://www.semanticweb.org/singer/k-pop/idol#>

SELECT ?4thboy
WHERE{?4thboy rdf:type kpop:4th;
rdf:type kpop:Boy_Group}
"""
qres = g.query(knows_query)

In [ ]:
qq = []
for row in qres:
  qq.append(list(row))

dff = pd.DataFrame(qq)

dff= dff.replace('http://www.semanticweb.org/singer/k-pop/idol#', '', regex=True)

dff

,0
0,스트레이키즈
1,투마로우바이투게더


노래 'antifragile'을 부른 그룹의 리더는?

In [ ]:
knows_query = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX kpop: <http://www.semanticweb.org/singer/k-pop/idol#>

SELECT ?leader
WHERE{kpop:antifragile kpop:song_of ?group
{?group kpop:leader ?leader}
}
"""
qres = g.query(knows_query)


In [ ]:
qq = []
for row in qres:
  qq.append(list(row))

dff = pd.DataFrame(qq)

dff= dff.replace('http://www.semanticweb.org/singer/k-pop/idol#', '', regex=True)

dff

,0
0,김채원


SM 소속 아이돌의 앨범과 그 앨범의 타이틀곡 찾기

In [ ]:
knows_query = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX kpop: <http://www.semanticweb.org/singer/k-pop/idol#>

SELECT ?idol ?album ?title
WHERE{?idol kpop:is_affiliated kpop:SM;
kpop:release ?album
{?album kpop:title_song ?title}}
"""
qres = g.query(knows_query)

In [ ]:
qq = []
for row in qres:
  qq.append(list(row))

dff = pd.DataFrame(qq)

dff= dff.replace('http://www.semanticweb.org/singer/k-pop/idol#', '', regex=True)

dff

,0,1,2
0,에스파,MY_WORLD,spicy
1,에스파,Next_Level,next_level


2015년도에 데뷔한 아이돌 찾기

In [ ]:
knows_query = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX kpop: <http://www.semanticweb.org/singer/k-pop/idol#>

SELECT ?idol
WHERE{?idol kpop:debut_year ?year
FILTER(?year = 2015)
}
"""
qres = g.query(knows_query)

In [ ]:
qq = []
for row in qres:
  qq.append(list(row))

dff = pd.DataFrame(qq)

dff= dff.replace('http://www.semanticweb.org/singer/k-pop/idol#', '', regex=True)

dff

,0
0,몬스타엑스
1,세븐틴


# SPARQL Quiz ⏰

##'꿈의 장'이라는 앨범의 타이틀 곡은?


In [ ]:
knows_query = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX kpop: <http://www.semanticweb.org/singer/k-pop/idol#>

SELECT ?y
WHERE{kpop:꿈의_장 kpop:title_song ?y}
"""
qres = g.query(knows_query)

In [ ]:
qq = []
for row in qres:
  qq.append(list(row))

dff = pd.DataFrame(qq)

dff= dff.replace('http://www.semanticweb.org/singer/k-pop/idol#', '', regex=True)

dff

,0
0,9와_4분의_3_승강장에서_너를_기다려


## 아이브의 데뷔 년도는?

In [ ]:
knows_query = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX kpop: <http://www.semanticweb.org/singer/k-pop/idol#>

SELECT ?year
WHERE{kpop:아이브 kpop:debut_year ?year;
}
"""
qres = g.query(knows_query)

In [ ]:
qq = []
for row in qres:
  qq.append(list(row))

dff = pd.DataFrame(qq)

dff= dff.replace('http://www.semanticweb.org/singer/k-pop/idol#', '', regex=True)

dff

,0
0,2021


## 4세대 걸그룹 모두 찾기

In [ ]:
knows_query = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX kpop: <http://www.semanticweb.org/singer/k-pop/idol#>

SELECT ?4thgirl
WHERE{?4thgirl rdf:type kpop:4th;
rdf:type kpop:Girl_Group}
"""
qres = g.query(knows_query)

In [ ]:
qq = []
for row in qres:
  qq.append(list(row))

dff = pd.DataFrame(qq)

dff= dff.replace('http://www.semanticweb.org/singer/k-pop/idol#', '', regex=True)

dff

,0
0,뉴진스
1,르세라핌
2,아이브
3,에스파


## 에스파 멤버 수와 방탄소년단 멤버 수의 합은?

In [ ]:
knows_query = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX kpop: <http://www.semanticweb.org/singer/k-pop/idol#>

SELECT ?에스파 ?BTS
WHERE{kpop:에스파 kpop:number_of_members ?에스파;
{kpop:방탄소년단 kpop:number_of_members ?BTS}
}
"""
qres = g.query(knows_query)

In [ ]:
qq = []
for row in qres:
  qq.append(list(row))

dff = pd.DataFrame(qq)

dff= dff.replace('http://www.semanticweb.org/singer/k-pop/idol#', '', regex=True)

dff

,0,1
0,4,7


## FML이라는 제목의 앨범을 낸 그룹과 그 앨범의 발매년도 찾기

In [ ]:
knows_query = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX kpop: <http://www.semanticweb.org/singer/k-pop/idol#>

SELECT ?group ?year
WHERE{?group kpop:release kpop:FML;
{kpop:FML kpop:release_year ?year}
}
"""
qres = g.query(knows_query)

In [ ]:
qq = []
for row in qres:
  qq.append(list(row))

dff = pd.DataFrame(qq)

dff= dff.replace('http://www.semanticweb.org/singer/k-pop/idol#', '', regex=True)

dff

,0,1
0,세븐틴,2023


## JYP 소속이면서 maniac이라는 노래를 부른 그룹의 리더는?

In [ ]:
knows_query = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX kpop: <http://www.semanticweb.org/singer/k-pop/idol#>

SELECT ?who
WHERE{?group kpop:is_affiliated kpop:JYP;
kpop:sing kpop:maniac;
kpop:leader ?who
}
"""
qres = g.query(knows_query)

In [ ]:
qq = []
for row in qres:
  qq.append(list(row))

dff = pd.DataFrame(qq)

dff= dff.replace('http://www.semanticweb.org/singer/k-pop/idol#', '', regex=True)

dff

,0
0,방찬


## 3세대 남자 아이돌이 발매한 앨범 중 2021년에 발매한 앨범의 이름과 타이틀곡

In [ ]:
knows_query = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX kpop: <http://www.semanticweb.org/singer/k-pop/idol#>

SELECT ?album ?song
WHERE{?group rdf:type kpop:3rd;
kpop:release ?album;
{?album kpop:release_year ?year;
kpop:title_song ?song
FILTER(?year = 2021)}
}

"""
qres = g.query(knows_query)

In [ ]:
qq = []
for row in qres:
  qq.append(list(row))

dff = pd.DataFrame(qq)

dff= dff.replace('http://www.semanticweb.org/singer/k-pop/idol#', '', regex=True)

dff

,0,1
0,NO_LIMIT,rush_hour
1,Butter,butter
